# Let's cook model

Let's combine what we've found so far.

- [What are ingredients?](https://www.kaggle.com/rejasupotaro/what-are-ingredients) (Preprocessing & Feature extraction)
- [Representations for ingredients](https://www.kaggle.com/rejasupotaro/representations-for-ingredients)

Steps are below.

1. Load dataset
2. Remove outliers
3. Preprocess
4. Create model
5. Check local CV
6. Train model
7. Check predicted values
8. Make submission

In [1]:
import json
import re
import unidecode
import numpy as np
import pandas as pd
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from tqdm import tqdm
tqdm.pandas()

## 1. Load dataset

In [2]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

## 2. Remove outliers

I saw weird recipes in the dataset .

- water => Japanese
- butter => Indian
- butter => French

Let's filter such single-ingredient recipes and see how it goes.

In [3]:
train['num_ingredients'] = train['ingredients'].apply(lambda x: len(x))
train = train[train['num_ingredients'] > 1]

## 3. Preprocess

Currently, the preprocess is like below.

- convert to lowercase
- remove hyphen
- remove numbers
- remove words which consist of less than 2 characters
- lemmatize

This process can be better.

In [4]:
lemmatizer = WordNetLemmatizer()
def preprocess(ingredients):
    ingredients_text = ' '.join(ingredients)
    ingredients_text = ingredients_text.lower()
    ingredients_text = ingredients_text.replace('-', ' ')
    words = []
    for word in ingredients_text.split():
        if re.findall('[0-9]', word): continue
        if len(word) <= 2: continue
        if '’' in word: continue
        word = lemmatizer.lemmatize(word)
        if len(word) > 0: words.append(word)
    return ' '.join(words)

for ingredient, expected in [
    ('Eggs', 'egg'),
    ('all-purpose flour', 'all purpose flour'),
    ('purée', 'purée'),
    ('1% low-fat milk', 'low fat milk'),
    ('half & half', 'half half'),
    ('safetida (powder)', 'safetida (powder)')
]:
    actual = preprocess([ingredient])
    assert actual == expected, f'"{expected}" is excpected but got "{actual}"'

In [5]:
train['x'] = train['ingredients'].progress_apply(lambda ingredients: preprocess(ingredients))
test['x'] = test['ingredients'].progress_apply(lambda ingredients: preprocess(ingredients))
train.head()

100%|██████████| 9944/9944 [00:01<00:00, 6928.18it/s]


,cuisine,id,ingredients,num_ingredients,x
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",9,romaine lettuce black olive grape tomato garli...
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",11,plain flour ground pepper salt tomato ground b...
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",12,egg pepper salt mayonaise cooking oil green ch...
3,indian,22213,"[water, vegetable oil, wheat, salt]",4,water vegetable oil wheat salt
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",20,black pepper shallot cornflour cayenne pepper ...


I need to tune the parameters of TfidfVectorizer later.

In [6]:
vectorizer = make_pipeline(
    TfidfVectorizer(sublinear_tf=True),
    FunctionTransformer(lambda x: x.astype('float16'), validate=False)
)

x_train = vectorizer.fit_transform(train['x'].values)
x_train.sort_indices()
x_test = vectorizer.transform(test['x'].values)

Encode cuisines to numeric values using LabelEncoder.

In [7]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train['cuisine'].values)
dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

{'brazilian': 0,
 'british': 1,
 'cajun_creole': 2,
 'chinese': 3,
 'filipino': 4,
 'french': 5,
 'greek': 6,
 'indian': 7,
 'irish': 8,
 'italian': 9,
 'jamaican': 10,
 'japanese': 11,
 'korean': 12,
 'mexican': 13,
 'moroccan': 14,
 'russian': 15,
 'southern_us': 16,
 'spanish': 17,
 'thai': 18,
 'vietnamese': 19}

In [8]:
from sklearn.model_selection import train_test_split
x, xt, y, yt = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

## 4. Create model

I've tried LogisticRegression, GaussianProcessClassifier, GradientBoostingClassifier, MLPClassifier, LGBMClassifier, SGDClassifier, Keras but SVC works better so far.

I need to take a look at models and the parameters more closely.

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
estimator = SVC(C=50,
                kernel='rbf',
                degree=3,
                gamma=1.57,
                coef0=1,
                shrinking = True,
                probability = False,
                max_iter=-1,
                decision_function_shape=None,
                random_state = None,
                cache_size=500)
# )
# classifier = ExtraTreesClassifier(n_estimators=500)
classifier = OneVsRestClassifier(estimator, n_jobs=-1)

## 5. Train model

If I become to be confident in the model, I train it with the whole train data for submission.

In [20]:
%%time
classifier.fit(x_train, y_train)

CPU times: user 1.06 s, sys: 271 ms, total: 1.33 s
Wall time: 12min 32s


OneVsRestClassifier(estimator=SVC(C=50, cache_size=500, class_weight=None, coef0=1,
  decision_function_shape=None, degree=3, gamma=1.57, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=-1)

## 6. Check predicted values

Check if the model fitted enough.

In [15]:
y_pred = label_encoder.inverse_transform(classifier.predict(x_train))
y_true = label_encoder.inverse_transform(y_train)

print(f'accuracy score on train data: {accuracy_score(y_true, y_pred)}')

def report2dict(cr):
    rows = []
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0: rows.append(parsed_row)
    measures = rows[0]
    classes = defaultdict(dict)
    for row in rows[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            classes[class_label][m.strip()] = float(row[j + 1].strip())
    return classes
report = classification_report(y_true, y_pred)
pd.DataFrame(report2dict(report)).T

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy score on train data: 0.9996478164620648


,f1-score,precision,recall,support
brazilian,1.0,1.0,1.0,467.0
british,1.0,1.0,1.0,804.0
cajun_creole,1.0,1.0,1.0,1546.0
chinese,1.0,1.0,1.0,2673.0
filipino,1.0,1.0,1.0,755.0
french,1.0,1.0,1.0,2644.0
greek,1.0,1.0,1.0,1174.0
indian,1.0,1.0,1.0,2997.0
irish,1.0,1.0,1.0,667.0
italian,1.0,1.0,1.0,7837.0


## 6. Make submission

It seems to be working well. Let's make a submission.

In [21]:
y_pred = label_encoder.inverse_transform(classifier.predict(x_test))
test['cuisine'] = y_pred
test[['id', 'cuisine']].to_csv('submission_svc(50，1.57).csv', index=False)
test[['id', 'cuisine']].head()

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,id,cuisine
0,18009,irish
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian


# DNN solution

In [9]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPool1D, GlobalAveragePooling1D, BatchNormalization, ActivityRegularization
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.cross_validation import train_test_split

random_seed = 47
Y = to_categorical(y_train, num_classes=20)
X_train, X_val, Y_train, Y_val = train_test_split(x_train, Y, test_size = 0.2, random_state=random_seed)


model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(2762,)))
# model.add(ActivityRegularization(l1=0.0, l2=0.2))
model.add(Dense(64, activation = "relu"))
# model.add(ActivityRegularization(l1=0.0, l2=0.2))
model.add(Dropout(0.2))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(20, activation = "softmax"))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=1e-04)
optimizer1 = Adam(lr=0.001, decay=1e-4)
optimizer2 = SGD(lr=0.1, decay=1e-4)
model.compile(optimizer = optimizer1 , loss = "categorical_crossentropy", metrics=["accuracy"])
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=0.01, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)
earlystopping = EarlyStopping(monitor='val_acc', patience=0.001, verbose=1, mode='auto')

history = model.fit(X_train, Y_train,
                      epochs=30,
                      batch_size=128,
                      validation_data=(X_val,Y_val),
                      verbose=1,callbacks=[learning_rate_reduction])

Train on 31801 samples, validate on 7951 samples
Epoch 1/30
31801/31801 [==============================] - 2s 67us/step - loss: 1.8563 - acc: 0.4633 - val_loss: 1.1459 - val_acc: 0.6619
Epoch 2/30
31801/31801 [==============================] - 2s 59us/step - loss: 1.1729 - acc: 0.6605 - val_loss: 0.9905 - val_acc: 0.7044
Epoch 3/30
31801/31801 [==============================] - 2s 61us/step - loss: 1.0021 - acc: 0.7075 - val_loss: 0.9196 - val_acc: 0.7231
Epoch 4/30
31801/31801 [==============================] - 2s 62us/step - loss: 0.8956 - acc: 0.7385 - val_loss: 0.8874 - val_acc: 0.7324
Epoch 5/30
31801/31801 [==============================] - 2s 62us/step - loss: 0.8049 - acc: 0.7645 - val_loss: 0.8788 - val_acc: 0.7491
Epoch 6/30
31801/31801 [==============================] - 2s 62us/step - loss: 0.7320 - acc: 0.7882 - val_loss: 0.8633 - val_acc: 0.7555
Epoch 7/30
31801/31801 [==============================] - 2s 63us/step - loss: 0.6668 - acc: 0.8078 - val_loss: 0.8645 - val_acc:

In [ ]:
preds = pd.DataFrame()
y_predicted = model.predict(x_test)
results = np.argmax(y_predicted,axis = 1)

y_pred = label_encoder.inverse_transform(results)
test['cuisine'] = y_pred
test[['id', 'cuisine']].to_csv('submission_dnn.csv', index=False)
test[['id', 'cuisine']].head()